In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from shutil import which
import time

# from selenium import webdriver

import requests
from lxml.html import fromstring
import time
import json
import os
import re

import pandas as pd

In [22]:
def scroll(t):
    height = driver.execute_script("return document.body.scrollHeight")
    for scrol in range(100,600,200):
        driver.execute_script(f"window.scrollTo(0,{scrol})")
        time.sleep(t)

In [23]:
def extract(page_source, start_sep, end_sep):
    source=page_source
#     start_sep='DIMENSIONS'
#     end_sep='MATERIAL'
    result=[]
    tmp=source.split(start_sep)
    for par in tmp:
        if end_sep in par:
            result.append(par.split(end_sep)[0])
#     result = result[3].replace('\\n', '\n')
#     result = "".join([s for s in result.strip().splitlines(True) if s.strip()])
    return result

def extract2(string, start, end):
    result = re.search('%s(.*)%s' % (start, end), string).group(1)
    return result

In [24]:
def empty_item():
    global product_details
    product_details = {
        'name': None,
        'sku': None,
        'gtin': None,
        'color': None,
        'finish': None,
        'price': None,
        'product_description': None,
        'technical_description': None,
        'display_dimensions': None,
        'length': None,
        'width': None,
        'height': None,
        'depth': None,
        'diameter': None,
        'weight': None,
        'arm_height': None,
        'seat_height': None,
        'table_clearance_height': None,
        'maximum_dimensions': None,
        'hero_image': None,
        'variant_image': None,
        'images': None,
        'files': None,
        'installations': None,
        'designer': None,
        'collection': None,
        'categories': None,
        'available_new': None,
        'made_in': None,
        'voltage': None,
        'bulb_type': None,
        'bulb_text': None,
        'canopy_dimensions': None,
        'ul_listed': None,
        'certifications': None,
        'warranty_&_care': None,
        'position': None,
        'notes': None,
        'url':  None
    }
    return product_details




In [41]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
# import requests
from lxml.html import fromstring
# from lxml.etree import tostring
import time
import json
import os

from copy import deepcopy


driver = webdriver.Firefox(executable_path=r'C:\Users\kchai\OneDrive\Documents\Moro\geckodriver-v0.31.0-win64\geckodriver')
driver.implicitly_wait(0)

driver.get("https://menudesignshop.com/collections/furniture")
try:
    welcome_path = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//button[@class = 'needsclick klaviyo-close-form kl-private-reset-css-Xuajs1']")))
    welcome_path.click()
except:
    pass
# scroll(0.2)
items = []
i = 1
exit = True
while exit:
    
    driver.get(f"https://menudesignshop.com/collections/furniture?page={i}")
    print(i)
    print('*\n' * 5)
    i+=1
    products = driver.find_elements(By.XPATH, "//div[@class = 'Grid__Cell 1/2--phone 1/3--tablet-and-up 1/4--lap-and-up']/div/div[@class = 'ProductItem__Wrapper']/a")
    if len(products)<1:
        exit = False
        break
    
    for product in products:
        empty_item()
        href = product.get_attribute("href")
        product_details['url'] = href
        print(href)
#         scroll(0.5)
#         hero_image = driver.find_element(By.XPATH, 
#         product_details['hero_image']  = hero_image
        driver.execute_script("window.open('" +href +"');")
        # switch to new window
        driver.switch_to.window(driver.window_handles[1])

        try:
            images_path = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[contains(@class , 'Product__SlideshowNavScroller')]/a")))
            images = []
            for img in images_path:
                images.append(img.get_attribute("href"))
            product_details['images'] = '\n'.join(images)
            product_details['uphostry'] = 0
        except:
            try:
                images_path = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[contains(@class, 'slick-slide')]/div/li/img")))
                images = []
                for img in images_path:
                    images.append('https:' +img.get_attribute("src"))
                product_details['images'] = '\n'.join(images)
                product_details['uphostry'] = 1
            except:
                product_details['images'] = ''


        response_product = requests.get(href)
        page_source_product = response_product.text
        info = json.loads(extract(page_source_product, '<script type="application/ld+json">', ' </script>')[0])
        product_details['name'] = info['name']
        try:
            product_details['gtin'] = info['gtin13']
        except:
            pass
        try:
            product_details['designer'] = info['brand']['name']
        except:
            pass
        try:
            product_details['product description'] = info['description'].split('\n[dimensions-start]')[0]
        except:
            pass
        try:
            product_details['category'] = info['category']
        except:
            pass
        try:
            product_details['hero_image'] = info['image']['image']
        except:
            pass
        try:
            if len(product_details['images']) <10:
                images = extract(page_source_product, '<img itemprop="image" src="', '"')[1:]
                product_details['images'] ='\n'.join(['https:' + i for i in images])
        except:
            pass
        
        try:
            pdfs_details = extract(page_source_product, '<ul>', '</ul>')[-1]
            if '.pdf' not in pdfs_details:
                pdfs_details = extract(page_source_product, '<ul>', '</ul>')[-2]
            pdf_files = extract(pdfs_details, 'href="', '"')
            pdf_files = [i for i in pdf_files if '.pdf' in i]
            pdf_names = extract(pdfs_details,  '>', '</')
            pdf_names = [i for i in pdf_names if len(i)>1 and '2D/3D' not in i]
            pdf_names = [i.title() for i in pdf_names]
            files = []
            for n, f in zip(pdf_names, pdf_files):
                files.append(n + ': ' + f)
            product_details['files'] = '\n'.join(files)   
            product_details['files'] = product_details['files'].replace(u'\xa0', u' ')
        except:
            pass
        
        varients = json.loads(extract2(page_source_product, 'var bcpo_product=', ';  var bcpo_settings'))
        tech = []
        desc = varients['description']
        dim = extract(desc, '[dimensions-start]', '[dimensions-end]')[0]
        product_details['display_dimensions'] = '\n'.join(extract(dim, '<li>', '</li>'))

        mat = extract(desc, '[materials-start]', '[materials-end]')[0]
        product_details['technical_description'] = '\n'.join(extract(mat, '<li>', '</li>'))
        
        
        uphosltry_options = driver.find_elements(By.XPATH, "//div[@class = 'card-body']/div[2]/div")
        
        if len(uphosltry_options) < 1:
            options = driver.find_elements(By.XPATH, "//div[@class = 'ProductForm__Variants']/div[@class = 'ProductForm__Option ProductForm__Option--labelled']")
            if len(options) == 1:
                option1 = options[0].find_elements(By.XPATH, './/ul/li')
                for opt1 in option1:
                    try:
                        opt1.click()
                    except:
                        pass
                    opt_name1 = opt1.find_element(By.XPATH, './/input').get_attribute('name')
                    opt_Value1 = opt1.find_element(By.XPATH, './/input').get_attribute('value')

                    try:
                        opt_img1 = opt1.find_element(By.XPATH, './/label').get_attribute('style')
                        product_details[f'option({opt_name1})'] = opt_Value1 + ': ' + 'https:' +extract(opt_img1, '(', ')')[0][1:-1]
                    except:
                        product_details[f'option({opt_name1})'] = opt_Value1

                    try:
                        sku_path = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//span[@class = 'ProductMeta__SkuNumber']")))
                        product_details['sku'] = sku_path.text
                    except:
                        product_details['sku'] = None

                    quantity_path = driver.find_element(By.XPATH, "//p[@class = 'ProductForm__Inventory Text--subdued']")
                    product_details['quantity'] = quantity_path.text
                    
                    try:
                        if len(product_details['quantity']) <3:
                            quantity_path = driver.find_element(By.XPATH, "//div[@class = 'Product__Backinstock Heading u-h6']")
                            product_details['quantity'] = quantity_path.text
                    except:
                        pass

                    try:
                        price_path = driver.find_element(By.XPATH, "//span[@class = 'ProductMeta__Price Price Price--compareAt Text--subdued u-h4']")
                        product_details['price'] = price_path.text

                    except:
                        price_path = driver.find_element(By.XPATH, "//span[@class = 'ProductMeta__Price Price Text--subdued u-h4']")
                        product_details['price'] = price_path.text

                    varient_path = driver.find_elements(By.XPATH, "//img[@class = 'ProductItem__Image Image--fadeIn regular-img lazyautosizes Image--lazyLoaded']")
                    variant_image = 'https:'+ varient_path[0].get_attribute('data-srcset').split(',')[-1][:-5]
                    product_details['variant_image'] = variant_image


                    print(product_details)
                    items.append(product_details.copy())

            elif len(options) == 2:
                option1 = WebDriverWait(options[0], 10).until(EC.visibility_of_all_elements_located((By.XPATH, './/ul/li')))
                option2 = WebDriverWait(options[1], 10).until(EC.visibility_of_all_elements_located((By.XPATH, './/ul/li')))
        #         print(option1)
        #         print(option2)
                for opt1 in option1:
                    time.sleep(1)
                    try:
                        opt1.click()
                    except:
                        pass

    #                 opt_img1 = opt1.get_attribute('data-option-tooltip-thumb')
                    product_details[f'option({opt_name1})'] = opt_Value1
                    for opt2 in option2:
                        try:
                            opt2.click()
                        except:
                            pass
                        if len(option1)>1:
                            opt_name1 = opt1.find_element(By.XPATH, './/input').get_attribute('name')
                            opt_Value1 = opt1.find_element(By.XPATH, './/input').get_attribute('value')
                            try:
                                opt_img1 = opt1.find_element(By.XPATH, './/label').get_attribute('style')
                                product_details[f'option({opt_name1})'] = opt_Value1 + ': ' + 'https:' +extract(opt_img1, '(', ')')[0][1:-1]
                            except:
                                product_details[f'option({opt_name1})'] = opt_Value1
                        if len(option2):
                            opt_name2 = opt2.find_element(By.XPATH, './/input').get_attribute('name')
                            opt_Value2 = opt2.find_element(By.XPATH, './/input').get_attribute('value')
            #                 opt_img1 = opt1.get_attribute('data-option-tooltip-thumb')
                            try:
                                opt_img2 = opt2.find_element(By.XPATH, './/label').get_attribute('style')
                                product_details[f'option({opt_name2})'] = opt_Value2 + ': ' + 'https:' +extract(opt_img2, '(', ')')[0][1:-1]
                            except:
                                product_details[f'option({opt_name2})'] = opt_Value2

                        try:
                            sku_path = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[@class = 'ProductMeta__SkuNumber']")))
                            product_details['sku'] = sku_path.text
                        except:
                            product_details['sku'] = None
                        try:
                            quantity_path = driver.find_element(By.XPATH, "//p[@class = 'ProductForm__Inventory Text--subdued']")
                            product_details['quantity'] = quantity_path.text
                        except:
                            pass
                        
                        try:
                            if len(product_details['quantity']) <3:
                                quantity_path = driver.find_element(By.XPATH, "//div[@class = 'Product__Backinstock Heading u-h6']")
                                product_details['quantity'] = quantity_path.text
                        except:
                            pass
                        
                        try:
                            price_path = driver.find_element(By.XPATH, "//span[@class = 'ProductMeta__Price Price Price--compareAt Text--subdued u-h4']")
                            product_details['price'] = price_path.text

                        except:
                            price_path = driver.find_element(By.XPATH, "//span[@class = 'ProductMeta__Price Price Text--subdued u-h4']")
                            product_details['price'] = price_path.text
                        try:
                            varient_path = driver.find_elements(By.XPATH, "//img[@class = 'ProductItem__Image Image--fadeIn regular-img lazyautosizes Image--lazyLoaded']")
                            variant_image = 'https:'+ varient_path[0].get_attribute('data-srcset').split(',')[-1][:-5]
                            product_details['variant_image'] = variant_image
                        except:
                            pass
                        
                        print(product_details)
                        items.append(product_details.copy())

            else:
                try:
                    sku_path = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[@class = 'ProductMeta__SkuNumber']")))
                    product_details['sku'] = sku_path.text
                except:
                    product_details['sku'] = None

                try:
                    price_path = driver.find_element(By.XPATH, "//span[@class = 'ProductMeta__Price Price Price--compareAt Text--subdued u-h4']")
                    product_details['price'] = price_path.text

                except:
                    try:
                        price_path = driver.find_element(By.XPATH, "//span[@class = 'ProductMeta__Price Price Text--subdued u-h4']")
                        product_details['price'] = price_path.text
                    except:
                        price_path = driver.find_element(By.XPATH, "//h3[@class = 'pro_price']/span[@class = 'product_price']")
                        product_details['price'] = price_path.get_attribute('innerHTML')
                try:
                    quantity_path = driver.find_element(By.XPATH, "//p[@class = 'ProductForm__Inventory Text--subdued']")
                    product_details['quantity'] = quantity_path.text
                except:
                    pass
                try:
                    if len(product_details['quantity']) <3:
                        quantity_path = driver.find_element(By.XPATH, "//div[@class = 'Product__Backinstock Heading u-h6']")
                        product_details['quantity'] = quantity_path.text
                except:
                    pass


                print(product_details)
                items.append(product_details.copy())
        else:
            options = driver.find_elements(By.XPATH, "//div[@class = 'selector-wrapper']")
            
            if len(options) == 1:
                option1 = options[0].find_elements(By.XPATH, './/select/option')
                for opt1 in option1:
                    try:
                        if opt1.text  != 'Select...':
                            try:
                                opt1.click()
                            except:
                                pass
                            opt_name1 = options[0].find_element(By.XPATH, './/label').get_attribute('innerHTML')
                            opt_Value1 = opt1.text
                            product_details[f'option({opt_name1})'] = opt_Value1
                    except:
                        pass
                    scroll(0.1)
                    uphosltry_options = driver.find_elements(By.XPATH, "//div[@class = 'card-body']/div[2]/div")
                    print(uphosltry_options)
                    for up_opt in uphosltry_options:
                        try:
                            up_opt.click()
                            up_opt_value = up_opt.get_attribute('data-family') + ' ' + up_opt.get_attribute('data-variant')
                            try:
                                opt_img = up_opt.find_element(By.XPATH, ".//div/img").get_attribute('src')
                                product_details['option(Upholstry)'] = up_opt_value+ ': '+ opt_img
                            except:
                                product_details['option(Upholstry)'] = up_opt_value
                        except:
                            print('uphosltry not clickable')
                            
#                         try:
#                             opt_name1 = options[0].find_element(By.XPATH, './/label').get_attribute('innerHTML')
#                             opt_Value1 = opt1.text
#                             product_details[f'option({opt_name1})'] = opt_Value1
#                         except:
#                             pass
                        
                        try:
                            sku_path = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[@id = 'currentSKU']")))
                            product_details['sku'] = sku_path.get_attribute('innerHTML')
                        except:
                            pass
                        
                        try:
                            quantity_path = driver.find_element(By.XPATH, "//span[@class = 'made-to-order']")
                            product_details['quantity'] = quantity_path.get_attribute('innerHTML')
                        except:
                            pass
                        
                        try:
                            price_path = driver.find_element(By.XPATH, "//h3[@class = 'pro_price']/s")
                            product_details['price'] = price_path.get_attribute('innerHTML')

                        except:
                            price_path = driver.find_element(By.XPATH, "//span[@class = 'product_price']")
                            product_details['price'] = price_path.get_attribute('innerHTML')
                            
                        if len(product_details['price'])<2:
                            price_path = driver.find_element(By.XPATH, "//span[@class = 'product_price']")
                            product_details['price'] = price_path.get_attribute('innerHTML')
                        try:
                            varient_path = driver.find_elements(By.XPATH, "//div[@class = 'slick-slide slick-current slick-active']/div/div/img")
                            variant_image = 'https:'+ varient_path[0].get_attribute('src')
                            product_details['variant_image'] = variant_image
                        except:
                            pass

                        print(product_details)
                        items.append(product_details.copy())

            elif len(options) == 2:
                option1 = WebDriverWait(options[0], 10).until(EC.visibility_of_all_elements_located((By.XPATH, './/select/option')))
                option2 = WebDriverWait(options[1], 10).until(EC.visibility_of_all_elements_located((By.XPATH, './/select/option')))
        #         print(option1)
        #         print(option2)
                for opt1 in option1:
                    try:
                        if opt1.text  != 'Select...':
                            try:
                                opt1.click()
                            except:
                                pass
                            opt_name1 = options[0].find_element(By.XPATH, './/label').get_attribute('innerHTML')
                            opt_Value1 = opt1.text
                            product_details[f'option({opt_name1})'] = opt_Value1
                    except:
                        pass

                    for opt2 in option2:
                        try:
                            if opt2.text  != 'Select...':
                                try:
                                    opt2.click()
                                except:
                                    pass
                                

                                opt_name2 = options[1].find_element(By.XPATH, './/label').get_attribute('innerHTML')
                                opt_Value2 = opt2.text
                                product_details[f'option({opt_name2})'] = opt_Value2
                        except:
                            pass
                        
                        scroll(0.1)
                        uphosltry_options = driver.find_elements(By.XPATH, "//div[@class = 'card-body']/div[2]/div")
                        print(uphosltry_options)
                        for up_opt in uphosltry_options:
                            try:
                                
                                up_opt.click()
                                up_opt_value = up_opt.get_attribute('data-family') + ' ' + up_opt.get_attribute('data-variant')
                                try:
                                    opt_img = up_opt.find_element(By.XPATH, ".//div/img").get_attribute('src')
                                    product_details['option(Upholstry)'] = up_opt_value + ': ' + opt_img
                                except:
                                    product_details['option(Upholstry)'] = up_opt_value 
                            except:
                                print('uphosltry not clickable')
                            try:
                                if opt1.text  != 'Select...':
                                    opt_name1 = options[0].find_element(By.XPATH, './/label').get_attribute('innerHTML')
                                    opt_Value1 = opt1.text
                                    product_details[f'option({opt_name1})'] = opt_Value1
                            except:
                                pass
                            
                            try:
                                if opt2.text  != 'Select...':
                                    opt_name2 = options[1].find_element(By.XPATH, './/label').get_attribute('innerHTML')
                                    opt_Value2 = opt2.text
                                    product_details[f'option({opt_name2})'] = opt_Value2
                            except:
                                pass
                            
                            try:
                                sku_path = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[@id = 'currentSKU']")))
                                product_details['sku'] = sku_path.get_attribute('innerHTML')
                            except:
                                pass

                            try:
                                quantity_path = driver.find_element(By.XPATH, "//span[@class = 'made-to-order']")
                                product_details['quantity'] = quantity_path.get_attribute('innerHTML')
                            except:
                                pass

                            try:
                                price_path = driver.find_element(By.XPATH, "//h3[@class = 'pro_price']/s")
                                product_details['price'] = price_path.get_attribute('innerHTML')

                            except:
                                price_path = driver.find_element(By.XPATH, "//span[@class = 'product_price']")
                                product_details['price'] = price_path.get_attribute('innerHTML')

                            if len(product_details['price'])<2:
                                price_path = driver.find_element(By.XPATH, "//span[@class = 'product_price']")
                                product_details['price'] = price_path.get_attribute('innerHTML')
                            try:   
                                varient_path = driver.find_elements(By.XPATH, "//div[@class = 'slick-slide slick-current slick-active']/div/div/img")
                                variant_image = 'https:'+ varient_path[0].get_attribute('src')
                                product_details['variant_image'] = variant_image
                            except:
                                pass

                            print(product_details)
                            items.append(product_details.copy())

            else:
                try:
                    sku_path = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//span[@id = 'currentSKU']")))
                    product_details['sku'] = sku_path.get_attribute('innerHTML')
                except:
                    pass

                try:
                    quantity_path = driver.find_element(By.XPATH, "//span[@class = 'made-to-order']")
                    product_details['quantity'] = quantity_path.get_attribute('innerHTML')
                except:
                    pass

                try:
                    price_path = driver.find_element(By.XPATH, "//h3[@class = 'pro_price']/s")
                    product_details['price'] = price_path.get_attribute('innerHTML')

                except:
                    price_path = driver.find_element(By.XPATH, "//span[@class = 'product_price']")
                    product_details['price'] = price_path.get_attribute('innerHTML')

                if len(product_details['price'])<2:
                    price_path = driver.find_element(By.XPATH, "//span[@class = 'product_price']")
                    product_details['price'] = price_path.get_attribute('innerHTML')

                print(product_details)
                items.append(product_details.copy())
        
        
        #close the new window
        driver.close()
        #back to main window
        driver.switch_to.window(driver.window_handles[0])
        

C:\Users\kchai\AppData\Local\Temp\ipykernel_668\3568515406.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'C:\Users\kchai\OneDrive\Documents\Moro\geckodriver-v0.31.0-win64\geckodriver')


1
*
*
*
*
*

https://menudesignshop.com/collections/furniture/products/plinth-bridge
{'name': 'Plinth Bridge', 'sku': '7040319', 'gtin': '5709262127405', 'color': None, 'finish': None, 'price': '$3,095', 'product_description': None, 'technical_description': '', 'display_dimensions': '', 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1024x.jpg?v=1664315989', 'variant_image': 'https: //cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1200x.jpg?v=1664315989 ', 'images': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1024x.jpg?v=1664315989\nhttps://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Pl

{'name': 'Plinth Bridge', 'sku': '7040530', 'gtin': '5709262127405', 'color': None, 'finish': None, 'price': '$2,395', 'product_description': None, 'technical_description': '', 'display_dimensions': '', 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1024x.jpg?v=1664315989', 'variant_image': 'https: //cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1200x.jpg?v=1664315989 ', 'images': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1024x.jpg?v=1664315989\nhttps://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Plinth_Low_Bridge_Offset_Sofa_Taper_Bowl_Flambeau_Table_1024x.jpg?v=1664943720\nhttps:/

{'name': 'Plinth Bridge', 'sku': '7040960', 'gtin': '5709262127405', 'color': None, 'finish': None, 'price': '$2,395', 'product_description': None, 'technical_description': '', 'display_dimensions': '', 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1024x.jpg?v=1664315989', 'variant_image': 'https: //cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1200x.jpg?v=1664315989 ', 'images': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_The_Seal_Lounge_Chair_Plinth_Bridge_Plinth_Low_1024x.jpg?v=1664315989\nhttps://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Plinth_Low_Bridge_Offset_Sofa_Taper_Bowl_Flambeau_Table_1024x.jpg?v=1664943720\nhttps:/

{'name': 'Brasilia, Lounge Chair, Sheepskin', 'sku': '8055002-001704ZZ', 'gtin': '5709262084050', 'color': None, 'finish': None, 'price': '$5,295', 'product_description': None, 'technical_description': '', 'display_dimensions': 'Lounge Chair: 33.10"H x 37.80"W x 32.30"D, Weight: 55.11 lbs.\xa0\nOttoman:\xa021.80<span data-mce-fragment="1">"H x 29.10"W x 15.70"D, Weight: 19 lbs.</span>\n', 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Brasilia_Lounge_Chair_Sheepskin_Brasilia_Ottoman_Plint_Tall_1024x.jpg?v=1650250546', 'variant_image': 'https: //cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Brasilia_Lounge_Chair_Sheepskin_Brasilia_Ottoman_Plint_Tall_1200x.jpg?v=1650250546 ', 'images': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Brasil

...

{'name': 'Marble Plinth', 'sku': '7020130', 'gtin': '5709262036196', 'color': None, 'finish': None, 'price': '$1,695', 'product_description': None, 'technical_description': 'Kunis Breccia Marble\nWhite (Carrara, from Italy)\nBlack (Nero Marquina, from Spain)\nGrey Brown (Kendzo, from Iran)\n<span>Rose (Calacatta Viola, from Italy)</span>\n<b>Marble is treated with a sealer for surface protection</b>', 'display_dimensions': 'Low: 10.2"H x 23.6"W x 39.3"L, Weight: 181 lbs.\nCubic: 15.8"H x 15.8"W x 15.8"L, Weight: 83 lbs.\nTall: 20.1"H x 11.8"W x 11.8"L, Weight: 72 lbs.\nLoad-bearing capacity: 66 lbs. / 30kg', 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Knitting_Chair_Plinth_Duca_Candle_Holder_Houkime_Rug_314a7847-a3a9-4a3d-a99f-ea4b419d5e91_1024x.jpg?v=1603392

{'name': 'Marble Plinth', 'sku': '7000319', 'gtin': '5709262036196', 'color': None, 'finish': None, 'price': '$2,895', 'product_description': None, 'technical_description': 'Kunis Breccia Marble\nWhite (Carrara, from Italy)\nBlack (Nero Marquina, from Spain)\nGrey Brown (Kendzo, from Iran)\n<span>Rose (Calacatta Viola, from Italy)</span>\n<b>Marble is treated with a sealer for surface protection</b>', 'display_dimensions': 'Low: 10.2"H x 23.6"W x 39.3"L, Weight: 181 lbs.\nCubic: 15.8"H x 15.8"W x 15.8"L, Weight: 83 lbs.\nTall: 20.1"H x 11.8"W x 11.8"L, Weight: 72 lbs.\nLoad-bearing capacity: 66 lbs. / 30kg', 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Knitting_Chair_Plinth_Duca_Candle_Holder_Houkime_Rug_314a7847-a3a9-4a3d-a99f-ea4b419d5e91_1024x.jpg?v=1603392

{'name': 'Marble Plinth', 'sku': '7000960', 'gtin': '5709262036196', 'color': None, 'finish': None, 'price': '$2,395', 'product_description': None, 'technical_description': 'Kunis Breccia Marble\nWhite (Carrara, from Italy)\nBlack (Nero Marquina, from Spain)\nGrey Brown (Kendzo, from Iran)\n<span>Rose (Calacatta Viola, from Italy)</span>\n<b>Marble is treated with a sealer for surface protection</b>', 'display_dimensions': 'Low: 10.2"H x 23.6"W x 39.3"L, Weight: 181 lbs.\nCubic: 15.8"H x 15.8"W x 15.8"L, Weight: 83 lbs.\nTall: 20.1"H x 11.8"W x 11.8"L, Weight: 72 lbs.\nLoad-bearing capacity: 66 lbs. / 30kg', 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Knitting_Chair_Plinth_Duca_Candle_Holder_Houkime_Rug_314a7847-a3a9-4a3d-a99f-ea4b419d5e91_1024x.jpg?v=1603392

{'name': 'Marble Plinth', 'sku': '7000530', 'gtin': '5709262036196', 'color': None, 'finish': None, 'price': '$2,395', 'product_description': None, 'technical_description': 'Kunis Breccia Marble\nWhite (Carrara, from Italy)\nBlack (Nero Marquina, from Spain)\nGrey Brown (Kendzo, from Iran)\n<span>Rose (Calacatta Viola, from Italy)</span>\n<b>Marble is treated with a sealer for surface protection</b>', 'display_dimensions': 'Low: 10.2"H x 23.6"W x 39.3"L, Weight: 181 lbs.\nCubic: 15.8"H x 15.8"W x 15.8"L, Weight: 83 lbs.\nTall: 20.1"H x 11.8"W x 11.8"L, Weight: 72 lbs.\nLoad-bearing capacity: 66 lbs. / 30kg', 'length': None, 'width': None, 'height': None, 'depth': None, 'diameter': None, 'weight': None, 'arm_height': None, 'seat_height': None, 'table_clearance_height': None, 'maximum_dimensions': None, 'hero_image': 'https://cdn.shopify.com/s/files/1/0075/3962/1924/products/MENU_Knitting_Chair_Plinth_Duca_Candle_Holder_Houkime_Rug_314a7847-a3a9-4a3d-a99f-ea4b419d5e91_1024x.jpg?v=1603392

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:436:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElement@chrome://remote/content/marionette/driver.sys.mjs:1480:15
despatch@chrome://remote/content/marionette/server.sys.mjs:303:40
execute@chrome://remote/content/marionette/server.sys.mjs:276:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:249:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:250:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:498:20


In [35]:
len(items)

98

In [36]:
import pandas as pd
import numpy as np

df = pd.DataFrame(items, )
df


,name,sku,gtin,color,finish,price,product_description,technical_description,display_dimensions,length,...,product description,category,display dimensions,quantity,variant image,option(Section),option(Upholstry),uphostry,option(option-0),option(option-1)
0,"Eave, Sectional Sofa",9952120-040300ZZ,5709262093311,None,None,"$1,409.95",None,PU Foam\nModules are secured to each other wit...,None,None,...,Inspired by architecture Eave Modular Sofa tak...,Sofa,"Pouf, 30in: 15.94""H x 29.53""W x<span><span> </...",Made To Order,https:https://cdn.shopify.com/s/files/1/0075/3...,NaN,NaN,NaN,NaN,NaN
1,"Eave, Sectional Sofa",9952120-040300ZZ,5709262093311,None,None,"$1,409.95",None,PU Foam\nModules are secured to each other wit...,None,None,...,Inspired by architecture Eave Modular Sofa tak...,Sofa,"Pouf, 30in: 15.94""H x 29.53""W x<span><span> </...",Made To Order,https:https://cdn.shopify.com/s/files/1/0075/3...,NaN,NaN,NaN,NaN,NaN
2,"Eave, Sectional Sofa",9952120-040300ZZ,5709262093311,None,None,"$1,409.95",None,PU Foam\nModules are secured to each other wit...,None,None,...,Inspired by architecture Eave Modular Sofa tak...,Sofa,"Pouf, 30in: 15.94""H x 29.53""W x<span><span> </...",Made To Order,https:https://cdn.shopify.com/s/files/1/0075/3...,NaN,NaN,NaN,NaN,NaN
3,"Eave, Sectional Sofa",9952120-040300ZZ,5709262093311,None,None,"$1,409.95",None,PU Foam\nModules are secured to each other wit...,None,None,...,Inspired by architecture Eave Modular Sofa tak...,Sofa,"Pouf, 30in: 15.94""H x 29.53""W x<span><span> </...",Made To Order,https:https://cdn.shopify.com/s/files/1/0075/3...,NaN,NaN,NaN,NaN,NaN
4,"Eave, Sectional Sofa",9952120-040300ZZ,5709262093311,None,None,"$1,409.95",None,PU Foam\nModules are secured to each other wit...,None,None,...,Inspired by architecture Eave Modular Sofa tak...,Sofa,"Pouf, 30in: 15.94""H x 29.53""W x<span><span> </...",Made To Order,https:https://cdn.shopify.com/s/files/1/0075/3...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Androgyne Side Table, 16.5in",1180539,5709262027514,None,None,$419.95,None,Powder-coated Steel\nCrystal Beige Rose Marble...,None,None,...,Distinguished by its playful yet elegant silho...,Side Table,"Base: <span>16.9""H x </span>15.75"" DIA\nTable ...",98 PIECES IN STOCK- SHIPS SOON,https: //cdn.shopify.com/s/files/1/0075/3962/1...,NaN,NaN,0.0,Black - All Metal: https://cdn.shopify.com/s/f...,NaN
94,"Androgyne Side Table, 16.5in",1108539U,5709262027514,None,None,$689.90,None,Powder-coated Steel\nCrystal Beige Rose Marble...,None,None,...,Distinguished by its playful yet elegant silho...,Side Table,"Base: <span>16.9""H x </span>15.75"" DIA\nTable ...",13 PIECES IN STOCK- SHIPS SOON,https: //cdn.shopify.com/s/files/1/0075/3962/1...,NaN,NaN,0.0,Black Marble Marquina: https://cdn.shopify.com...,NaN
95,"Androgyne Side Table, 16.5in",1108989U,5709262027514,None,None,$689.90,None,Powder-coated Steel\nCrystal Beige Rose Marble...,None,None,...,Distinguished by its playful yet elegant silho...,Side Table,"Base: <span>16.9""H x </span>15.75"" DIA\nTable ...",35 PIECES IN STOCK- SHIPS SOON,https: //cdn.shopify.com/s/files/1/0075/3962/1...,NaN,NaN,0.0,Emperador Dark Marble: https://cdn.shopify.com...,NaN
96,Afteroom Stool,8480530,5709262024391,None,None,$199.95,None,"Powder-Coated Welded Steel, Painted MDF",None,None,...,A study of unity between functionalism and min...,Chair,"Dining Stool: 18.5""H x 17.6"" DIA, Weight: 7.1 ...",,https: //cdn.shopify.com/s/files/1/0075/3962/1...,NaN,NaN,0.0,Dining Height (18.5in),Black Painted MDF: https://cdn.shopify.com/s/f...


In [38]:
df['option(Upholstry)'].unique()

array([nan,
       'MenuBoucle 02: http://resources.menudesignshop.com/apis/Images/Textile/MenuBoucle/MenuBoucle-02.png',
       'MenuBoucle 06: http://resources.menudesignshop.com/apis/Images/Textile/MenuBoucle/MenuBoucle-06.png',
       'Savanna 202: http://resources.menudesignshop.com/apis/Images/Textile/Savanna/Savanna-202.png',
       'MenuBoucle 08: http://resources.menudesignshop.com/apis/Images/Textile/MenuBoucle/MenuBoucle-08.png',
       'MenuBoucle 16: http://resources.menudesignshop.com/apis/Images/Textile/MenuBoucle/MenuBoucle-16.png',
       'Safire 012: http://resources.menudesignshop.com/apis/Images/Textile/Safire/Safire-012.png',
       'CityVelvet CA7832-031: http://resources.menudesignshop.com/apis/Images/Textile/CityVelvet/CityVelvet-CA7832-031.png'],
      dtype=object)

In [99]:
for i in range(len(df)):
    if len(df['quantity'][i]) <2:
        print(i)

106
130
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
232
380
469
615
616
617
618


TypeError: object of type 'float' has no len()

In [62]:
df.to_excel('menu_final.xlsx', index = False)

In [16]:
df = pd.read_excel(r"C:\Users\kchai\Downloads\menu_upt.xlsx")
df

,name,sku,gtin,url,price,quantity,uphostry,option(option-0),option(Style),option(Finish),...,made_in,voltage,bulb_type,bulb_text,canopy_dimensions,ul_listed,certifications,warranty_&_care,position,notes
0,Plinth Bridge,7040319,5.709262e+12,https://menudesignshop.com/collections/furnitu...,"$3,095",18 PIECES IN STOCK- SHIPS SOON,0.0,Calacatta Viola Marble: https://cdn.shopify.co...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Plinth Bridge,7040630,5.709262e+12,https://menudesignshop.com/collections/furnitu...,"$2,395",15 PIECES IN STOCK- SHIPS SOON,0.0,White Carrara Marble: https://cdn.shopify.com/...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Plinth Bridge,7040530,5.709262e+12,https://menudesignshop.com/collections/furnitu...,"$2,395",7 PIECES IN STOCK- SHIPS SOON,0.0,Black Marquina Marble: https://cdn.shopify.com...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Plinth Bridge,7040130,5.709262e+12,https://menudesignshop.com/collections/furnitu...,"$2,895",1 PIECE IN STOCK.- SHIPS SOON,0.0,Kunis Breccia Marble: https://cdn.shopify.com/...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Plinth Bridge,7040960,5.709262e+12,https://menudesignshop.com/collections/furnitu...,"$2,395",2 PIECES IN STOCK- SHIPS SOON,0.0,Grey Kendzo Marble: https://cdn.shopify.com/s/...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,Wire Base,4771539,5.709263e+12,https://menudesignshop.com/collections/accesso...,$59.95,57 PIECES IN STOCK- SHIPS SOON,0.0,Medium,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1046,Wire Base,4772539,5.709263e+12,https://menudesignshop.com/collections/accesso...,$74.95,48 PIECES IN STOCK- SHIPS SOON,0.0,Large,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1047,"Wire Pot, Black",4774539,5.709263e+12,https://menudesignshop.com/collections/accesso...,$74.95,69 PIECES IN STOCK- SHIPS SOON,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048,Yana Brewing Pot,4470139,5.709262e+12,https://menudesignshop.com/collections/accesso...,$119.95,110 PIECES IN STOCK- SHIPS SOON,0.0,Grey Glazed: https://cdn.shopify.com/s/files/1...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
names = list(df['name'].unique())
names

['Plinth Bridge',
 'Brasilia, Lounge Chair, Sheepskin',
 'Eave, Sectional Sofa',
 'Marble Plinth',
 'Turning Table',
 'Oval Wall Mirror',
 'Androgyne Side Table, 16.5in',
 'Afteroom Stool',
 'Umanoff Side Table',
 'Knitting Chair, Sheepskin Upholstery',
 'Rail Desk',
 'Bath Floor Mirror, Rectangular',
 'NoNo Table',
 'Bath Wall Mirror, Rectangular',
 'Co Dining Chair, Plastic',
 'Androgyne Lounge Table, Stone/Marble',
 'Co Chair, Non-Upholstered, Dining Height',
 'Afteroom Chair, Non-Upholstered, Black',
 'Houkime Rug',
 'Nimbus Mirror, Round',
 'Harbour Column Table, Round Table Top',
 'Harbour Side Chair, Dining Height, Hard Shell',
 'Passage Stool',
 'Passage Lounge Table',
 'Corbel Shelf',
 'Co Lounge Chair',
 'Co Lounge Chair, Fully Upholstered',
 'Septembre Coffee Table',
 'Offset Sofas',
 'Marble Plinth, Low',
 'Afteroom Bench',
 'Rail Shelf',
 'Harbour Side Chair, Dining Height, Upholstered',
 'Harbour Arm Chair, Dining Height, Hard Shell',
 'Afteroom Plus Chair, Upholstered',


In [25]:
df['hero_image_upd'] = None
df_upt = pd.DataFrame()
for name in names:
    df_temp = df[df['name'] == name]

    df_temp['hero_image_upd'].iloc[0] = df_temp['hero_image'].iloc[0]    

    df_upt = df_upt.append(df_temp, ignore_index=True)

C:\Users\kchai\AppData\Local\Temp\ipykernel_16616\3002511423.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['hero_image_upd'].iloc[0] = df_temp['hero_image'].iloc[0]
C:\Users\kchai\AppData\Local\Temp\ipykernel_16616\3002511423.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_upt = df_upt.append(df_temp, ignore_index=True)


In [27]:
df_upt.to_excel('menu_new.xlsx', index = False)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    1050 non-null   object 
 1   sku                     1001 non-null   object 
 2   gtin                    830 non-null    float64
 3   url                     1050 non-null   object 
 4   price                   1024 non-null   object 
 5   quantity                1006 non-null   object 
 6   uphostry                627 non-null    float64
 7   option(option-0)        603 non-null    object 
 8   option(Style)           12 non-null     object 
 9   option(Finish)          6 non-null      object 
 10  option(Arms)            32 non-null     object 
 11  option(Size)            13 non-null     object 
 12  option(Type)            64 non-null     object 
 13  option(Upholstry)       418 non-null    object 
 14  option(Section)         111 non-null    

In [28]:
df['product_description'].unique()

array(['Plinth Bridge has a classic silhouette and modern material appeal. Designed for use independently as well as in combination with the Plinth Grand and Plinth Low podiums – forming a bridge across the low-slung designs for elevated surface space – the sleek design is offered in a range of beautiful, natural materials: from classic White Carrera marble and the intriguing purple-veined Calacatta Viola to sand-colored Kunis Breccia stone.',
       'Scandinavian mid-century design aesthetics and Brazilian Modernism meet on the low-lying Brasilia Lounge Chair, designed for MENU by Anderssen &amp; Voll. The luxurious armchair-lounge chair hybrid offers unparalleled sitting comfort thanks to its robust frame and soft upholstery. Characterized by its stout and sturdy legs, made using traditional joinery techniques, and exaggerated armrests and cushioning seat and backrest, upholstered in a range of soft fabrics, it creates a sense of comfort and ease in any setting.',
       'Inspired by

In [8]:
for i in range(len(df)):
    
    try:
        df['display_dimensions'][i] = re.sub('<[^>]+>', '', df['display_dimensions'][i])
    except:
        print(i)
        pass
        
    try:
        df['technical_description'][i] = re.sub('<[^>]+>', '', df['technical_description'][i])
    except:
        print(i)
        pass


0
0
1
1
2
2
3
3
4
4
5
6
7
8
9
135
136
137
138
139
140
343
344
507
507
508
508
509
509
510
510
511
511
512
512
513
513
514
514


C:\Users\kchai\AppData\Local\Temp\ipykernel_16616\3284968017.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['display_dimensions'][i] = re.sub('<[^>]+>', '', df['display_dimensions'][i])
C:\Users\kchai\AppData\Local\Temp\ipykernel_16616\3284968017.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['technical_description'][i] = re.sub('<[^>]+>', '', df['technical_description'][i])


797
797
798
798
799
799
800
800
801
801
802
802
803
803
804
804
805
805
903
903
904
904
905
905
906
906
907
907
1122
1123
1124
1125
1194
1195
1196
1197
1198
1199
1200
1201
1207


In [10]:
import numpy as np
df = df.astype(object).replace(np.nan, None)

In [11]:
for i in range(len(df)):
    if (df['images'][i] is None) or (len(df['images'][i])<5):
        df['images'][i] = df['hero_image'][i]

In [13]:
for i in range(len(df)):
    if len(df['images']) <5:
        print(i)

In [14]:
df.to_excel('menu_upt.xlsx', index = False)

In [5]:
df_all = pd.read_excel(r"C:\Users\chaitanya\Downloads\menu_all.xlsx")
df_all

,name,sku,price,url,images,gtin,designer,product description,category,hero_image,...,categories,collection,voltage,bulb_type,bulb_text,UL Listed,certifications,warranty & care,position,notes
0,Hashira High Floor Lamp,1507699U,749.95,https://menudesignshop.com/collections/lightin...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,Norm Architects,Set on an elegant powder-coated steel foot for...,Floor Lamp,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hashira High Floor Lamp,1507059U,749.95,https://menudesignshop.com/collections/lightin...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,Norm Architects,Set on an elegant powder-coated steel foot for...,Floor Lamp,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hashira Wall Lamp,1506699U,239.95,https://menudesignshop.com/collections/lightin...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,Norm Architects,"NOTE: In the US, lamps are wall mounted withou...",Wall Lamp,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hashira Wall Lamp,1506059U,239.95,https://menudesignshop.com/collections/lightin...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,Norm Architects,"NOTE: In the US, lamps are wall mounted withou...",Wall Lamp,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MENU Brand Book - A Sense of Community,2021.0,0.0,https://menudesignshop.com/collections/lightin...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,MENU,"Brand book is free with any purchase, and ship...",Brand Book,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3484,"Knitting Chair, Sheepskin Upholstery",9680006-030E02ZZ,4594.95,https://menudesignshop.com/collections/furnitu...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,Ib Kofod-Larsen,Please Note: Sheepskin is a natural material a...,Lounge Chair,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3485,"Knitting Chair, Sheepskin Upholstery",9680006-020D02ZZ,4594.95,https://menudesignshop.com/collections/furnitu...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,Ib Kofod-Larsen,Please Note: Sheepskin is a natural material a...,Lounge Chair,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3486,"Knitting Chair, Sheepskin Upholstery",9682006-000C02ZZ,5129.95,https://menudesignshop.com/collections/furnitu...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,Ib Kofod-Larsen,Please Note: Sheepskin is a natural material a...,Lounge Chair,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3487,Rail Shelf,1207039.0,219.95,https://menudesignshop.com/collections/furnitu...,https://cdn.shopify.com/s/files/1/0075/3962/19...,5.709262e+12,Keiji Ashizawa,"The Rail Shelf, created for MENU by Japanese d...",Wall Shelf,https://cdn.shopify.com/s/files/1/0075/3962/19...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_price = pd.read_excel(r"C:\Users\chaitanya\Downloads\MENU 2022 Q3 Price List - MSRP (USD) (1).xlsx")
df_price

,Mfr / Vendor SKU,Old SKU,Product Name,Q3 2022 MSRP (USD),Q2 2022 MSRP (USD),Price Change (Y/N)
0,4411002,-,"Abacus, Candle Holder, Height - 1 inch",33.95,33.95,No
1,4412002,-,"Abacus, Candle Holder, Height - 2 inch",43.95,43.95,No
2,4413002,-,"Abacus, Candle Holder, Height - 3 inch",49.95,49.95,No
3,4730949,-,"Aer , Bowl, Finish - Smoke",199.95,219.95,Yes
4,4736949,-,"Aer , Vase, Finish - Smoke, Height - 13 inch",249.95,269.95,Yes
...,...,...,...,...,...,...
2081,7354849,-,"Zet, Shelf, Size - 60 inch, Finish - Dark Stai...",160.00,160.00,No
2082,7354039,-,"Zet, Shelf, Size - 60 inch, Finish - Natural Oak",160.00,160.00,No
2083,Total,NaN,NaN,NaN,NaN,NaN
2084,NaN,NaN,NaN,NaN,NaN,NaN
